In [88]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
import datetime
import re

In [89]:
def get_next_page(driver):
    try:
        driver.find_element_by_class_name("ui_button.nav.next.primary").send_keys(Keys.ENTER)
        return True
    except:
        print('마지막 페이지입니다')
        return False

In [90]:
def convert_date(date):
    stay_date = date.split('숙박 날짜: ')
    stay_date = stay_date[1]

    # 20XX년 X월 형태의 str
    stay_date = stay_date.replace('년', '').replace(' ', '').replace('월', '')
    dt = datetime.datetime.strptime(stay_date, '%Y%m')
    dt = dt.date() #20XX-XX-01 형태의 date 객체
    return dt

In [23]:
# convert_date 실행 예시
convert_date('숙박 날짜: 2020년 11월')

datetime.date(2020, 11, 1)

In [91]:
def get_data(driver):
    iteration = True
    df = pd.DataFrame(columns = ['nickname', 'score', 'review', 'stay_date'])
    try:
        time.sleep(20)
        #WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, 
        #                                                                    "div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu")))
        # 더보기 클릭
        driver.find_element_by_css_selector("div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu").click()

        time.sleep(5)
        # 숙박 날짜 가져오기
        dates = driver.find_elements_by_class_name("_34Xs-BQm")
        stay_dates = []
        for d in dates:
            stay_date = convert_date(d.text)
            stay_dates.append(stay_date)
            if stay_date < datetime.date(2019,7,1):
                iteration = False

        # 닉네임 가져오기
        nicknames = driver.find_elements_by_css_selector("div._310S4sqz > div > div._2fxQ4TOx > span > a")
        # 별점 가져오기
        rating_codes = driver.find_elements_by_css_selector("div.oETBfkHU > div._2UEC-y30 > div > span")
        scores = []
        for rc in rating_codes:
            cls_attr = str(rc.get_attribute("class")).split("ui_bubble_rating bubble_")
            score = int(cls_attr[1])
            scores.append(score)

        # 리뷰 가져오기
        reviews = driver.find_elements_by_css_selector("div.oETBfkHU > div._3hDPbqWO > div._2f_ruteS._1bona3Pu > div.cPQsENeY > q > span")
        
        for i in range(len(nicknames)):
            df.loc[i] = [nicknames[i].text, scores[i], reviews[i].text, stay_dates[i]]
            #print(df.loc[i])

    except ElementClickInterceptedException as e:
        print(e)
    except StaleElementReferenceException as e:
        print(e)

    finally:
        return df, iteration

In [92]:
def hotel_review_crawling(url):
    # 데이터 수집
    options = webdriver.ChromeOptions()
    options.add_experimental_option("excludeSwitches", ["enable-logging"])
    driver = webdriver.Chrome(options=options)

    data = pd.DataFrame(columns = ['nickname', 'score', 'review', 'stay_date'])

    #첫페이지 시작
    #데이터 크롤링, df에 추가하기
    #다음페이지 클릭 -> disbaled 나오기 전까지 반복

    driver.get(url)
    driver.implicitly_wait(10)
    iteration = True
    page = 1
    while iteration:
        print(page, '번째 페이지')
        tmp_df, iteration = get_data(driver)
        data = pd.concat([data, tmp_df], axis=0, ignore_index=True)
        if iteration == False:
            break
        iteration = get_next_page(driver) 
        page += 1

    #data.to_csv('./hotel_review_test_sillastay_mapo.csv', encoding="utf-8-sig")
    
    return data

In [93]:
def preprocessing_data(data):
    # 개행문자 제거
    for idx, review in enumerate(data['review']):
        data['review'][idx] = review.replace('\n', ' ')
    # 날짜순으로 내림차순 정렬
    data.sort_values(by='stay_date', ascending=False, 
                     ignore_index=True, inplace=True)
    data = data[data['stay_date'] > datetime.date(2019,7,1)]
    # 특수문자, 이모지 제거
    ###
    return data

In [94]:
# 실제로 실행하는 코드
url = "https://www.tripadvisor.co.kr/Hotel_Review-g294197-d9452203-Reviews-Shilla_Stay_Gwanghwamun-Seoul.html"
url2 = "https://www.tripadvisor.co.kr/Hotel_Review-g294197-d8118189-Reviews-or240-Shilla_Stay_Mapo-Seoul.html"
url3 = "https://www.tripadvisor.co.kr/Hotel_Review-g294197-d7054545-Reviews-Shilla_Stay_Yeoksam-Seoul.html"

data_sample = hotel_review_crawling(url3)
data_sample

1 번째 페이지
2 번째 페이지
3 번째 페이지
4 번째 페이지
5 번째 페이지
6 번째 페이지
7 번째 페이지
8 번째 페이지
9 번째 페이지
10 번째 페이지
11 번째 페이지
12 번째 페이지
13 번째 페이지
14 번째 페이지
15 번째 페이지
16 번째 페이지
17 번째 페이지
18 번째 페이지


,nickname,score,review,stay_date
0,Park,50,시설도 좋도 깨끗하고 서비스가 너무 좋다 높은방으로 배정해줘서 뷰도 굉장히 좋았다 ...,2021-06-01
1,Mochi,50,직원들 모두 친절하시고 룸 상태도 깨끗하고 조용해서 좋습니다. 무엇보다도 위치부분에...,2021-06-01
2,이가현,50,"방도 깨끗하고, 직원들도 너무 친절했습니다\n무엇보다 호텔의 위치가 너무 좋아서 서...",2021-05-01
3,jakekim,50,직원들도 너무 친정하고 깕끔해ㅛㅓ 가성비 그자체\n강남에 이만한 호텔은 없는것 같습...,2021-05-01
4,비갠오후,40,며칠전 예약 후 오늘 점심뷔페 잘 먹고 왔습니다. 인테리어도 상당히 깔끔하고 북적거...,2021-03-01
...,...,...,...,...
85,yonghanyi1,50,깔끔하네요... 중심가라서 좋고...\n가성비 좋고... 친절하고... 분위도 괜찮...,2019-10-01
86,JAEMIN S,40,업무때문에 1박을 했습니다. 방이 크진 않지만 신라스테이 브랜드 이름답게 깔끔하고 ...,2018-12-01
87,minga,50,처음 방문했어요! 강남에 일이 있어서 친구랑 방문했는데 깔끔한 침구류와 깔끔한 욕실...,2019-09-01
88,cookki,50,업무상 며칠 묵었습니다 대로변에 있어서 찾기 쉽더군요\n\n깔끔하고 식사도 좋네요 ...,2019-08-01


In [96]:
# 중간에 크롤링이 끊겼을 경우
url_next = "https://www.tripadvisor.co.kr/Hotel_Review-g294197-d7054545-Reviews-or90-Shilla_Stay_Yeoksam-Seoul.html"
data_sample2 = hotel_review_crawling(url_next)

data_sample_concat = pd.concat([data_sample, data_sample2], axis=0, ignore_index=True)

1 번째 페이지
2 번째 페이지
3 번째 페이지
4 번째 페이지
5 번째 페이지


In [97]:
data_sample_concat

,nickname,score,review,stay_date
0,Park,50,시설도 좋도 깨끗하고 서비스가 너무 좋다 높은방으로 배정해줘서 뷰도 굉장히 좋았다 ...,2021-06-01
1,Mochi,50,직원들 모두 친절하시고 룸 상태도 깨끗하고 조용해서 좋습니다. 무엇보다도 위치부분에...,2021-06-01
2,이가현,50,"방도 깨끗하고, 직원들도 너무 친절했습니다\n무엇보다 호텔의 위치가 너무 좋아서 서...",2021-05-01
3,jakekim,50,직원들도 너무 친정하고 깕끔해ㅛㅓ 가성비 그자체\n강남에 이만한 호텔은 없는것 같습...,2021-05-01
4,비갠오후,40,며칠전 예약 후 오늘 점심뷔페 잘 먹고 왔습니다. 인테리어도 상당히 깔끔하고 북적거...,2021-03-01
...,...,...,...,...
110,Pioneer18802352158,50,깨끗하고 친절하고 좋아요.\n위치도 좋고 강남 중심이어서 다니기도\n편하고 주변에 ...,2019-07-01
111,레오채,40,늦은새벽 체크인에도 직원분의 친절한 안내 감사했습니다‥\n크지 않은 객실이지만 깔끔...,2019-07-01
112,Yoonmi,40,혼자서 묵을 숙소를 찾았는데 시설도 좋고 충분히 늦은 시간에도 직원분이 친절하셔서 ...,2019-07-01
113,lunamong,50,방은 깨끗하고 침구도 편안합니다\n예전에도 한번 왔었는데\n그때도 만족했고 이번엔 ...,2019-05-01


In [69]:
# 기본 전처리 전 원본 데이터 저장(유실방지)
data_sample.to_csv('./data/hotel_review_shillastay_yeoksam_origin.csv', encoding="utf-8-sig")

In [98]:
data = preprocessing_data(data_sample_concat)

In [99]:
data

,nickname,score,review,stay_date
0,Park,50,시설도 좋도 깨끗하고 서비스가 너무 좋다 높은방으로 배정해줘서 뷰도 굉장히 좋았다 ...,2021-06-01
1,Mochi,50,직원들 모두 친절하시고 룸 상태도 깨끗하고 조용해서 좋습니다. 무엇보다도 위치부분에...,2021-06-01
2,이가현,50,"방도 깨끗하고, 직원들도 너무 친절했습니다 무엇보다 호텔의 위치가 너무 좋아서 서울...",2021-05-01
3,jakekim,50,직원들도 너무 친정하고 깕끔해ㅛㅓ 가성비 그자체 강남에 이만한 호텔은 없는것 같습니...,2021-05-01
4,비갠오후,40,며칠전 예약 후 오늘 점심뷔페 잘 먹고 왔습니다. 인테리어도 상당히 깔끔하고 북적거...,2021-03-01
...,...,...,...,...
91,Nexwho,50,휴가로 하루밤을 묵었습니다. 호텔은 역삼역과 선릉역 사이에 위치하고 있어 교통이 ...,2019-08-01
92,Lee seokhee,50,무료 룸 업그레이드도 받고 객실도 깔끔하고 전체적으로 만족스러웠습니다. 배달 음식 ...,2019-08-01
93,cookki,50,업무상 며칠 묵었습니다 대로변에 있어서 찾기 쉽더군요 깔끔하고 식사도 좋네요 특히...,2019-08-01
94,Kimwonseob,50,정말 오랜만에 이정도 가격에 가성비에 최고인가같아요!!!짱이에여 다음에 또방문할거같...,2019-08-01


In [100]:
# csv 저장을 위한 date to str convert
data['stay_date'] = data['stay_date'].apply(lambda X : X.strftime('%Y-%m-%d'))
# 기본 전처리후 데이터 csv 파일로 저장 
data.to_csv('./data/hotel_review_shillastay_yeoksam.csv', encoding="utf-8-sig")

<ipython-input-100-0d95ffe4274f>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['stay_date'] = data['stay_date'].apply(lambda X : X.strftime('%Y-%m-%d'))


In [101]:
# 리뷰 내 이모지 제거
data['review'] = data['review'].apply(lambda X : X.encode('euc-kr', 'ignore').decode('euc-kr'))
data.to_csv('./data/hotel_review_shillastay_yeoksam_del_emoji.csv', encoding="utf-8-sig")

<ipython-input-101-a9a0689142a1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['review'] = data['review'].apply(lambda X : X.encode('euc-kr', 'ignore').decode('euc-kr'))


In [103]:
# 리뷰 내 특수문자 제거
test_data = data
special = re.compile(r'[^ A-Za-z0-9가-힣+]')
test_data['review'] = [special.sub('', s) for s in test_data['review']]

# 특수문자 제거 후 파일 저장
test_data.to_csv('./data/hotel_review_shillastay_yeoksam_final.csv', encoding="utf-8-sig")

<ipython-input-103-a2b534a793de>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['review'] = [special.sub('', s) for s in test_data['review']]


In [104]:
test_data

,nickname,score,review,stay_date
0,Park,50,시설도 좋도 깨끗하고 서비스가 너무 좋다 높은방으로 배정해줘서 뷰도 굉장히 좋았다 ...,2021-06-01
1,Mochi,50,직원들 모두 친절하시고 룸 상태도 깨끗하고 조용해서 좋습니다 무엇보다도 위치부분에서...,2021-06-01
2,이가현,50,방도 깨끗하고 직원들도 너무 친절했습니다 무엇보다 호텔의 위치가 너무 좋아서 서울에...,2021-05-01
3,jakekim,50,직원들도 너무 친정하고 깕끔해 가성비 그자체 강남에 이만한 호텔은 없는것 같습니다 ...,2021-05-01
4,비갠오후,40,며칠전 예약 후 오늘 점심뷔페 잘 먹고 왔습니다 인테리어도 상당히 깔끔하고 북적거리...,2021-03-01
...,...,...,...,...
91,Nexwho,50,휴가로 하루밤을 묵었습니다 호텔은 역삼역과 선릉역 사이에 위치하고 있어 교통이 편...,2019-08-01
92,Lee seokhee,50,무료 룸 업그레이드도 받고 객실도 깔끔하고 전체적으로 만족스러웠습니다 배달 음식 시...,2019-08-01
93,cookki,50,업무상 며칠 묵었습니다 대로변에 있어서 찾기 쉽더군요 깔끔하고 식사도 좋네요 특히...,2019-08-01
94,Kimwonseob,50,정말 오랜만에 이정도 가격에 가성비에 최고인가같아요짱이에여 다음에 또방문할거같아용 ...,2019-08-01
